## Code de l'an passé pour le tracking du Thymio en live

Déjà que c'est pas simple à mettre en place, alors si en plus faut le faire chacun de son côté chez soi ... ^^'

On peut y rester ouvert ceci-dit :)

In [3]:
import cv2
import time
import math

import numpy as np

%matplotlib notebook 
import matplotlib.pyplot as plt
%matplotlib notebook

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from tqdm import tqdm

import IPython.display as Disp
from ipywidgets import widgets

In [7]:
scale=150
cam = cv2.VideoCapture(0) #Open webcam stream
while True:
                ret_val, image = cam.read() #Read 
                #get the webcam size
                height, width, channels = image.shape
                #prepare the crop
                centerX,centerY=int(height/2),int(width/2)
                radiusX,radiusY= int(scale*height/100),int(scale*width/100)

                minX,maxX=centerX-radiusX,centerX+radiusX
                minY,maxY=centerY-radiusY,centerY+radiusY

                cropped = image[minX:maxX, minY:maxY]
                resized_cropped = cv2.resize(cropped, (width, height)) 

                cv2.imshow('Webcam', resized_cropped)
                if cv2.waitKey(1) == 27: 
                    break  # esc to quit

                #add + or - 5 % to zoom

                if cv2.waitKey(1) == ord('p'): 
                    scale += 5  # +5
                    print(scale)
                    if (radiusX >= height or radiusY >= width):
                        radiusX=height
                        radiusY=width
                        
                    
                if cv2.waitKey(1) == ord('m'): 
                    scale -= 5  # +5
        #showpic = cv2.imwrite("environment.png",image)
        #print(showpic)
cam.release()
cv2.destroyAllWindows()

# tracker with initial selection

### listing of the trackers

In [8]:
def computeCenter(p1, p2):
    x1,y1 = p1[0],p1[1]
    x2,y2 = p2[0],p2[1]
    
    xc = (x1 + x2)/2
    yc = (y1 + y2)/2
    
    center = [xc,yc]
    
    return center
    

In [9]:
from random import randint
 
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
 
def createTrackerByName(trackerType):
    # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)

    return tracker

## loading the video

In [10]:
# Create a video capture object to read videos
cap = cv2.VideoCapture(0)

if (not cap.isOpened()):
    print("fkjnr")


time.sleep(2)
# Read first frame
success, frame = cap.read()


width, height = frame.shape[1], frame.shape[0]

data = "this is going to move to somewhere great"
%store data
# quit if unable to read the video file

if not success:
    print('Failed to read video')
    sys.exit(1)
cap.release()

Stored 'data' (str)


## selection of the frames

In [11]:
## Select boxes
bboxes = []
colors = [] 
 
# OpenCV's selectROI function doesn't work for selecting multiple objects in Python
# So we will call this function in a loop till we are done selecting all objects
i = 0

while True:
    print("Ready to select new ROI")
    i += 1
    # draw bounding boxes over objects
    # selectROI's default behaviour is to draw box starting from the center
    # when fromCenter is set to false, you can draw box starting from top left corner
    bbox = cv2.selectROI('MultiTracker', frame)
    bboxes.append(bbox)
    colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    print("Object {}, bbox {}".format(i, bbox))
    print("Press **q** to quit selecting boxes and start tracking")
    print("Press any other key to select next object \n")
    k = cv2.waitKey(0) & 0xFF
    if (k == 113):  # q is pressed
        break

print('Selected bounding boxes {}'.format(bboxes))
cv2.destroyAllWindows()

Ready to select new ROI
Object 1, bbox (10, 177, 36, 37)
Press **q** to quit selecting boxes and start tracking
Press any other key to select next object 

Ready to select new ROI
Object 2, bbox (78, 186, 24, 23)
Press **q** to quit selecting boxes and start tracking
Press any other key to select next object 

Selected bounding boxes [(10, 177, 36, 37), (78, 186, 24, 23)]


## tracker set up

In [12]:
# Instead of MIL, you can also use
# BOOSTING, KCF, TLD, MEDIANFLOW or GOTURN
# Specify the tracker type

trackerType = "BOOSTING" 
 
# Create MultiTracker object
multiTracker = cv2.MultiTracker_create()
 
# Initialize MultiTracker 
for bbox in bboxes:
    multiTracker.add(createTrackerByName(trackerType), frame, bbox)

## running the video with tracker display

In [ ]:
output_notebook()
cap = cv2.VideoCapture(0)
# Creating the plot
p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
show(p, notebook_handle=True)
CenterThym=[]
FrontThym=[]

# Process video and track objects
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(frame)

    # draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
        center = computeCenter(p1,p2)
        if i==0 :
            cthym = center
        else :
            fthym = center
    anglethym = math.atan2((fthym[1]-cthym[1]),(fthym[0]-cthym[0]))
    #anglethym = math.acos(anglethym)
    #CenterThym.append([cthym[0],cthym[1]])
    #FrontThym.append([fthym[0],fthym[1]])
    posThym = [cthym[0], cthym[1], anglethym]
    %store posThym
    print(posThym)

    # show frame
    #cv2.imshow('MultiTracker', frame)
    frame=cv2.cvtColor(frame[::-1, :,:], cv2.COLOR_BGR2RGBA)# because Bokeh expects a RGBA image and flips images vertically
    myImage.data_source.data['image']=[frame]
    push_notebook()
    time.sleep(0.01)
    # quit on ESC button
    if cv2.waitKey(1) & 0xFF == 27:  # Esc pressed
        break

# When everything done, release the video capture object
cap.release()

print("Done displaying the video")

Loading BokehJS ...

Stored 'posThym' (list)
[18.0, 180.5, 0.11242713083019014]
Stored 'posThym' (list)
[27.0, 194.5, -0.03277514440407577]
Stored 'posThym' (list)
[28.0, 192.5, 0.06548002492346482]
Stored 'posThym' (list)
[27.0, 193.5, 0.04834938665190287]
Stored 'posThym' (list)
[27.0, 193.5, 0.04834938665190287]
Stored 'posThym' (list)
[28.0, 194.5, 0.03277514440407577]
Stored 'posThym' (list)
[27.0, 193.5, 0.04834938665190287]
Stored 'posThym' (list)
[28.0, 191.5, 0.081784380617996]
Stored 'posThym' (list)
[27.0, 193.5, 0.04834938665190287]
Stored 'posThym' (list)
[28.0, 192.5, 0.06548002492346482]
Stored 'posThym' (list)
[28.0, 190.5, 0.09804527729383924]
Stored 'posThym' (list)
[28.0, 192.5, 0.06548002492346482]
Stored 'posThym' (list)
[27.0, 193.5, 0.04834938665190287]
Stored 'posThym' (list)
[28.0, 192.5, 0.06548002492346482]
Stored 'posThym' (list)
[28.0, 191.5, 0.081784380617996]
Stored 'posThym' (list)
[28.0, 191.5, 0.081784380617996]
Stored 'posThym' (list)
[28.0, 191.5, 0.081784380617996]
Stor

[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (list)
[29.0, 190.5, 0.09966865249116202]
Stored 'posThym' (lis

Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.11809202841909398]
Stored

Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThym' (list)
[29.0, 188.5, 0.1347712844777311]
Stored 'posThy

...

[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stor

[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stor

[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[407.0, 404.5, -0.3805063771123649]
Stor

[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stor

[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stored 'posThym' (list)
[408.0, 404.5, -0.4636476090008061]
Stor

In [57]:
cap.release()